# 🏃‍♀️ クイックスタート

Weaveの機能を理解していきましょう！

# <a href="https://colab.research.google.com/drive/1bdymP7p7d4z7izsS-PhMUxXcD38p9Hqr" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colabで開く"/></a>

## 🪄 `weave`ライブラリのインストールとログイン


ライブラリをインストールし、アカウントにログインすることから始めましょう。

この例では、W&B Inferenceを使用するため、Public Cloud環境のWANDB_API_KEYが必要です。Dedicated CloudやOnpremiseを利用されている方は、Public Cloudのkeyも用意してください。

In [ ]:
import os
import wandb
import weave
import openai
import json
#========================================
# 環境変数を適切に設定してください
#========================================
# os.environ["WANDB_BASE_URL"] = "https://api.wandb.ai"
os.environ["WANDB_API_KEY"] = ""
os.environ["WANDB_API_KEY_PUBLIC_CLOUD"] = ""  # Public cloudユーザーはWANDB_API_KEYと同じ値を使用して問題ないです。

wandb.login()
PROJECT = "<entity>/weave-handson" # handsonで利用するprojectです。entityのところを置き換えてください。
INFERENCE_PROJECT="<entity>/<project>" # Inferenceの利用料トラックのために利用するProjectです。詳しくはREADMEをご確認ください。
weave.init(PROJECT)

# 関数の入力と出力をトラッキング

Weaveにより、ユーザーは関数呼び出しを、コード、入力、出力、さらにはLLMトークンとコストまでトラッキングできます。以下のセクションでは次の内容をカバーします：

* カスタム関数
* ベンダー統合
* ネストした関数呼び出し
* エラートラッキング

## カスタム関数のトラッキング

トラッキングしたい関数に@weave.opデコレータを追加します

In [ ]:
@weave.op()
def echo(user_input):
    return user_input + " " + user_input

result = echo("hello")
print(result)

上記の👆 wandbリンクをクリックすることで、インタラクティブダッシュボードを見つけることができます。

`weave.op`を追加して関数を呼び出した後、リンクにアクセスして、プロジェクト内でトラッキングされているのを確認してください。

💡 コードを自動的にトラッキングしています。コードタブをご覧ください！

## Integrationを利用したトラッキング（W&B Inference、OpenAI、Anthropic、Mistralなど）

ここでは、`W&B Inference`への全ての呼び出しを自動的にトラッキングしています。Weaveは、多くのLLMライブラリを@weaveなしで自動的にトラッキングしてくれます。

In [ ]:
# W&B Inference clientの初期化
# Public cloudユーザーはWANDB_API_KEYと同じ値をWANDB_API_KEY_PUBLIC_CLOUDとして使用してください
client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY_PUBLIC_CLOUD") or os.getenv("WANDB_API_KEY"),
    project=INFERENCE_PROJECT,
)
model_name="openai/gpt-oss-20b"

In [ ]:
# Trace the model call in Weave
def run_chat():
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "ジョークを言ってください。"}
        ],
    )
    return response.choices[0].message.content

# Run and log the traced call
output = run_chat()
print(output)

## Nested関数のトラッキング

基本を確認したので、上記のすべてを組み合わせて、Nested関数をトラッキングしてみましょう。


In [ ]:
@weave.op()
def correct_grammar(user_input):
    echoed_input = echo(user_input)
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Please create a song by following the user input.",
            },
            {"role": "user", "content": echoed_input},
        ],
        temperature=0,
    )
    return response.choices[0].message.content


result = correct_grammar("hello")
print(result)

## エラーのトラッキング

コードがクラッシュするたびに、weaveは問題の原因をハイライトします。LLMレスポンスからデータを解析する際に時々発生するJSONパースエラーなどを見つけるのに特に有用です。

In [ ]:
@weave.op()
def correct_grammar(user_input):
    echoed_input = echo(user_input)
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Please create a song by following the user input.",
            },
            {"role": "user", "content": echoed_input},
        ],
        temperature=0,
        response_format={"type": "json_object"},
    )
    return json.loads(response.choices[0].message.content)


result = correct_grammar("hello")

# Traceの高度なTips


### サンプリングレートの制御

@weave.opデコレータのtracing_sample_rateパラメータを設定することで、opの呼び出しがトレースされる頻度を制御できます。これは、呼び出しのサブセットのみをトレースする必要がある高頻度opに有用です。

サンプリングレートはルート呼び出しにのみ適用されることに注意してください。opにサンプルレートがあっても、最初に別のopによって呼び出された場合、そのサンプリングレートは無視されます。

In [ ]:
@weave.op(tracing_sample_rate=0.1)  # Only trace ~10% of calls
def high_frequency_op(x: int) -> int:
    return x + 1

@weave.op(tracing_sample_rate=1.0)  # Always trace (default)
def always_traced_op(x: int) -> int:
    return x + 1

### 呼び出し表示名

In [ ]:
# Decorate your function
@weave.op
def my_function(name: str):
    return f"Hello, {name}!"

# Call your function -- Weave will automatically track inputs and outputs
print(my_function("World"))

In [ ]:
# 1st method
result = my_function("World", __weave={"display_name": "My Custom Display Name"})

In [ ]:
# 2nd method
result, call = my_function.call("World")
call.set_display_name("My Custom Display Name")


In [ ]:
# 3rd method
@weave.op(call_display_name="My Custom Display Name")
def my_function(name: str):
    return f"Hello, {name}!"

my_function("World")


### PIIの編集

一部の組織では、大規模言語モデル（LLM）ワークフローで名前、電話番号、メールアドレスなどの個人識別情報（PII）を処理します。このデータをWeights & Biases（W&B）Weaveに保存することは、コンプライアンスとセキュリティのリスクをもたらします。

機密データ保護機能により、トレースがWeaveサーバーに送信される前に、個人識別情報（PII）を自動的に編集できます。この機能はMicrosoft PresidioをWeave Python SDKに統合しており、SDK レベルで編集設定を制御できることを意味します。

[詳細ドキュメント](https://weave-docs.wandb.ai/guides/tracking/redact-pii)

# オブジェクトのトラッキング

Weaveでは、システムプロンプトや使用しているモデル、データセットなどのアセットを`weave.Objects`内でバージョン管理できます。

## プロンプトトラッキング

In [ ]:
# StringPrompt1
system_prompt = weave.StringPrompt("You are a pirate")
weave.publish(system_prompt, name="pirate_prompt")

response = client.chat.completions.create(
  model=model_name,
  messages=[
    {
      "role": "system",
      "content": system_prompt.format()
    },
    {
      "role": "user",
      "content": "Explain general relativity in one paragraph."
    }
  ],
)

In [ ]:
# StringPrompt2
system_prompt = weave.StringPrompt("Talk like a pirate. I need to know I'm listening to a pirate.")
weave.publish(system_prompt, name="pirate_prompt")

response = client.chat.completions.create(
  model=model_name,
  messages=[
    {
      "role": "system",
      "content": system_prompt.format()
    },
    {
      "role": "user",
      "content": "Explain general relativity in one paragraph."
    }
  ],
)


In [ ]:
# MessagesPrompt1
prompt = weave.MessagesPrompt([
    {
        "role": "system",
        "content": "You are a stegosaurus, but don't be too obvious about it."
    },
    {
        "role": "user",
        "content": "What's good to eat around here?"
    }
])
weave.publish(prompt, name="dino_prompt")

response = client.chat.completions.create(
  model="gpt-4o",
  messages=prompt.format(),
)

In [ ]:
# parameterizing prompts
prompt = weave.StringPrompt("Solve the equation {equation}")
weave.publish(prompt, name="calculator_prompt")


response = client.chat.completions.create(
  model=model_name,
  messages=[
    {
      "role": "user",
      "content": prompt.format(equation="1 + 1 = ?")
    }
  ],
)

## モデルトラッキング


In [ ]:
class WandBInferenceGrammarCorrector(weave.Model):
    # プロパティは完全にユーザー定義
    model_name: str
    system_message: str

    @weave.op()
    def predict(self, user_input):

        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": user_input},
            ],
        )
        return response.choices[0].message.content

corrector = WandBInferenceGrammarCorrector(
    model_name=model_name,
    system_message="You are a grammar checker, correct the following user input.",
)


result = corrector.predict("     That was so easy, it was a piece of pie!       ")
print(result)

## データセットトラッキング


In [ ]:
dataset = weave.Dataset(
    name="grammar-correction",
    rows=[
        {
            "user_input": "   That was so easy, it was a piece of pie!   ",
            "expected": "That was so easy, it was a piece of cake!",
        },
        {"user_input": "  I write good   ",
         "expected": "I write well"},
        {
            "user_input": "  GPT-3 is smartest AI model.   ",
            "expected": "GPT-3 is the smartest AI model.",
        },
    ],
)

weave.publish(dataset)

## Retrieve Published Objects & Ops


In [ ]:
ref_url = ""
prompt = weave.ref(ref_url).get()

print(prompt)

# オフライン評価



## 方法1: [標準メソッド](https://weave-docs.wandb.ai/guides/core-types/evaluations)
予測と評価の両方をサンプルごとに実行し、評価をします。

In [ ]:
# Method1
import weave
from weave import Evaluation

# Our dataset has "input_text" but our model expects "question"
examples = [
    {"input_text": "What is the capital of France?", "expected": "Paris"},
    {"input_text": "Who wrote 'To Kill a Mockingbird'?", "expected": "Harper Lee"},
    {"input_text": "What is the square root of 64?", "expected": "8"},
]

@weave.op()
def preprocess_example(example):
    # Rename input_text to question
    return {
        "question": example["input_text"]
    }

@weave.op()
def match_score(expected: str, output: dict) -> dict:
    return {'match': expected == output['generated_text']}

@weave.op()
def function_to_evaluate(question: str):
    return {'generated_text': f'Answer to: {question}'}

# Create evaluation with preprocessing
evaluation = weave.Evaluation(
    dataset=examples,
    scorers=[match_score],
    preprocess_model_input=preprocess_example
)

# Initialize the Weave project
weave.init(PROJECT)

# In Jupyter/Colab, use 'await' instead of 'asyncio.run'
async def run_eval():
    await evaluation.evaluate(function_to_evaluate)

await run_eval()

## 方法2: [EvaluationLogger](https://weave-docs.wandb.ai/guides/evaluation/evaluation_logger)
バッチ予測が適用できます。

In [ ]:
import weave
from weave.flow.eval_imperative import EvaluationLogger

# Initialize the logger with optional metadata
eval_logger = EvaluationLogger(
    model="my_local_model",
    dataset="my_dataset"
)

# Example input data
eval_samples = [
    {'inputs': {'a': 1, 'b': 2}, 'expected': 3},
    {'inputs': {'a': 2, 'b': 3}, 'expected': 5},
    {'inputs': {'a': 3, 'b': 4}, 'expected': 7},
]

# Local model logic: simply add the numbers
@weave.op
def user_model(a: int, b: int) -> int:
    return a + b

# Evaluate each sample
for sample in eval_samples:
    inputs = sample["inputs"]
    model_output = user_model(**inputs)  # Call model with unpacked input

    # Log the prediction
    pred_logger = eval_logger.log_prediction(
        inputs=inputs,
        output=model_output
    )

    # Compare output with expected value
    expected = sample["expected"]
    correctness_score = model_output == expected
    pred_logger.log_score(
        scorer="correctness",
        score=correctness_score
    )

    # Finalize log for this prediction
    pred_logger.finish()

# Log overall evaluation summary
summary_stats = {"subjective_overall_score": 1.0}
eval_logger.log_summary(summary_stats)

print("Evaluation logging complete. View results in the Weave UI.")


# オンライン評価

# フィードバック

In [ ]:
client = weave.init(PROJECT)
call = client.get_call("") #@param

# Adding an emoji reaction
call.feedback.add_reaction("👍")

# Adding a note
call.feedback.add_note("this is a note")

# Adding custom key/value pairs.
# The first argument is a user-defined "type" string.
# Feedback must be JSON serializable and less than 1 KB when serialized.
call.feedback.add("correctness", { "value": 5 })

# Scorers

In [ ]:
import weave
from weave import Scorer


class LengthScorer(Scorer):
    @weave.op
    def score(self, output: str) -> dict:
        """A simple scorer that checks output length."""
        return {
            "length": len(output),
            "is_short": len(output) < 100
        }

@weave.op
def generate_text(prompt: str) -> str:
    return "Hello, world!"

# Get both result and Call object
result, call = generate_text.call("Say hello")

# Now you can apply scorers
await call.apply_scorer(LengthScorer())

## ガードレールとしてのScorersの使用
ガードレールは、LLM出力がユーザーに到達する前に実行される安全チェックとして機能します。実用的な例を以下に示します：

In [ ]:
import weave
from weave import Scorer
import asyncio
import nest_asyncio  # Required for running asyncio in Google Colab

# Apply nest_asyncio to avoid event loop issues in Google Colab
nest_asyncio.apply()

# ==== 1. Define text generation function ====
@weave.op
def generate_text(prompt: str) -> str:
    """Simulated LLM text generation (basic logic)."""
    responses = {
        "hello": "Hello! How can I help you?",
        "bad": "You are terrible!",  # Example of toxic response
        "good": "You are wonderful!",
    }
    return responses.get(prompt.lower(), "I don't understand your request.")

# ==== 2. Define the Toxicity Scorer ====
class ToxicityScorer(Scorer):
    @weave.op
    def score(self, output: str) -> dict:
        """
        Evaluate the generated content for toxic language.
        """
        toxic_words = {"terrible", "hate", "stupid"}  # Simple keyword-based detection
        flagged = any(word in output.lower() for word in toxic_words)

        return {
            "flagged": flagged,
            "reason": "Detected toxic language" if flagged else None
        }

# ==== 3. Function to generate safe responses ====
async def generate_safe_response(prompt: str) -> str:
    # Generate text using LLM
    result, call = generate_text.call(prompt)

    # Apply toxicity scoring
    safety = await call.apply_scorer(ToxicityScorer())

    # If flagged as toxic, return a warning message
    if safety.result["flagged"]:
        return f"I cannot generate that content: {safety.result['reason']}"

    return result

# ==== 4. Run test cases ====
async def main():
    prompts = ["hello", "bad", "good"]

    for prompt in prompts:
        response = await generate_safe_response(prompt)
        print(f"Prompt: {prompt}\nResponse: {response}\n")

# Run the async function in Google Colab
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


## モニターとしてのScorersの使用

モニターは、操作をブロックすることなく、時間の経過とともに品質メトリクスを追跡するのに役立ちます。これは以下に有用です：

* 品質トレンドの特定
* モデルドリフトの検出
* モデル改善のためのデータ収集





In [ ]:
import weave
from weave import Scorer
import json
import xml.etree.ElementTree as ET
import random
import asyncio
import nest_asyncio  # Required for Google Colab

# Apply nest_asyncio to avoid event loop issues in Google Colab
nest_asyncio.apply()

# ==== 1. Define Text Generation Function ====
@weave.op
def generate_text(prompt: str) -> str:
    """Simulated LLM response generation."""
    if prompt.lower() == "json":
        return '{"message": "Hello, world!"}'  # Valid JSON
    elif prompt.lower() == "xml":
        return "<message>Hello, world!</message>"  # Valid XML
    else:
        return "Generated response..."

# ==== 2. Custom Scorer for JSON Validation ====
class CustomJSONScorer(Scorer):
    @weave.op
    def score(self, output: str) -> dict:
        """Check if the output is valid JSON."""
        try:
            json.loads(output)
            return {"valid_json": True}
        except json.JSONDecodeError:
            return {"valid_json": False}

# ==== 3. Custom Scorer for XML Validation ====
class CustomXMLScorer(Scorer):
    @weave.op
    def score(self, output: str) -> dict:
        """Check if the output is valid XML."""
        try:
            ET.fromstring(output)
            return {"valid_xml": True}
        except ET.ParseError:
            return {"valid_xml": False}

# ==== 4. Function to Generate Response with Monitoring ====
async def generate_with_monitoring(prompt: str) -> str:
    """
    Generates a response and applies monitoring (randomly 10% of the time).
    """
    # Generate text and capture tracking info
    result, call = generate_text.call(prompt)

    # Sample monitoring (only apply scorers to 10% of calls)
    if random.random() < 0.1:
        # Apply manual scorers asynchronously
        json_score = await call.apply_scorer(CustomJSONScorer())
        xml_score = await call.apply_scorer(CustomXMLScorer())

        print(f"Monitoring Applied - JSON Valid: {json_score.result['valid_json']}, XML Valid: {xml_score.result['valid_xml']}")

    return result

# ==== 5. Run Test Cases ====
async def main():
    prompts = ["json", "xml", "text"]

    for prompt in prompts:
        response = await generate_with_monitoring(prompt)
        print(f"Prompt: {prompt}\nResponse: {response}\n")

# Run the async function in Google Colab
loop = asyncio.get_event_loop()
loop.run_until_complete(main())



# メディア

## 画像

In [ ]:
import weave
from openai import OpenAI
import requests
from PIL import Image

weave.init(project_name=PROJECT)
client = OpenAI()

@weave.op()
def generate_image(prompt: str) -> Image:
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url
    image_response = requests.get(image_url, stream=True)
    image = Image.open(image_response.raw)

    # return a PIL.Image.Image object to be logged as an image
    return image

image = generate_image("a cat with a pumpkin hat")

## 音声

In [ ]:
import weave
from openai import OpenAI
import wave

weave.init(project_name=PROJECT)
client = OpenAI()

@weave.op
def make_audio_file_streaming(text: str) -> wave.Wave_read:
    with client.audio.speech.with_streaming_response.create(
        model="tts-1",
        voice="alloy",
        input=text,
        response_format="wav",
    ) as res:
        res.stream_to_file("output.wav")

    # return a wave.Wave_read object to be logged as audio
    return wave.open("output.wav")

make_audio_file_streaming("Hello, how are you? What did you do yesterday?")

## 動画

In [ ]:
import time
from google import genai
from google.genai import types
from moviepy.editor import VideoFileClip, ColorClip, VideoClip
import weave

weave.init(PROJECT)

@weave.op()
def store_videos(name, client, generated_video):
  client.files.download(file=generated_video.video)
  generated_video.video.save(f"new_video{name}.mp4")  # save the video

@weave.op()
def save_videos_in_weave(video_path):
    VideoFileClip(video_path, has_mask=False, audio=True)
    new_clip = clip.subclip(0, 1)
    return new_clip

@weave.op()
def generate_videos(prompt):
    client = genai.Client()  # read API key from GOOGLE_API_KEY
    operation = client.models.generate_videos(
        model="veo-2.0-generate-001",
        prompt="Panning wide shot of a calico kitten sleeping in the sunshine",
        config=types.GenerateVideosConfig(
            person_generation="dont_allow",  # "dont_allow" or "allow_adult"
            aspect_ratio="16:9",  # "16:9" or "9:16"
            ),
        )
    
    while not operation.done:
        time.sleep(10)
        operation = client.operations.get(operation)
    
    for n, generated_video in enumerate(operation.response.generated_videos):
        client.files.download(file=generated_video.video)
        generated_video.video.save(f"video{n}.mp4")  # save the video
        save_videos_in_weave(f"video{n}.mp4")
        
 
generate_videos("Panning wide shot of a calico kitten sleeping in the sunshine")
